In [20]:
from __future__ import print_function
import IPython
import sys
from music21 import *
import numpy as np
from grammar import *
from qa import *
from preprocess import * 
from music_utils import *
from data_utils import *
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

In [21]:
IPython.display.Audio('./data/30s_seq.mp3')

In [22]:
X, Y, n_values, indices_values = load_music_utils()
print('shape of X:', X.shape)
print('number of training examples:', X.shape[0])
print('Tx (length of sequence):', X.shape[1])
print('total # of unique values:', n_values)
print('Shape of Y:', Y.shape)

shape of X: (60, 30, 78)
number of training examples: 60
Tx (length of sequence): 30
total # of unique values: 78
Shape of Y: (30, 60, 78)


In [23]:
n_a = 64

In [24]:
reshapor = Reshape((1, 78))                        # Used in Step 2.B of djmodel(), below
LSTM_cell = LSTM(n_a, return_state = True)         # Used in Step 2.C
densor = Dense(n_values, activation='softmax')     # Used in Step 2.D

In [25]:

def djmodel(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data 
    
    Returns:
    model -- a keras model with the 
    """
    
    # Define the input of your model with a shape 
    X = Input(shape=(Tx, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    ### START CODE HERE ### 
    # Step 1: Create empty list to append the outputs while you iterate (≈1 line)
    outputs = list()
    
    # Step 2: Loop
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = Lambda(lambda x: x[:, t, :])(X)
        # Step 2.B: Use reshapor to reshape x to be (1, n_values) (≈1 line)
        x = reshapor(x)
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(x, initial_state = [a, c])
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        # Step 2.E: add the output to "outputs"
        outputs.append(out)
        
    # Step 3: Create model instance
    model = Model(inputs = [X, a0, c0], outputs = outputs)
    
    ### END CODE HERE ###
    
    return model

In [26]:
model = djmodel(Tx = 30 , n_a = 64, n_values = 78)

In [27]:
# Check your model
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 30, 78)       0                                            
__________________________________________________________________________________________________
lambda_81 (Lambda)              (None, 78)           0           input_3[0][0]                    
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 1, 78)        0           lambda_81[0][0]                  
                                                                 lambda_82[0][0]                  
                                                                 lambda_83[0][0]                  
                                                                 lambda_84[0][0]            

In [28]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
m = 60
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

In [30]:
model.fit([X, a0, c0], list(Y), epochs=100)

Epoch 1/100
60/60 [==============================] - 10s 159ms/step - loss: 125.7305 - dense_2_loss: 0.0000e+00 - dense_2_accuracy: 0.0167 - dense_2_accuracy_1: 0.0000e+00 - dense_2_accuracy_2: 0.0833 - dense_2_accuracy_3: 0.0167 - dense_2_accuracy_4: 0.0500 - dense_2_accuracy_5: 0.0333 - dense_2_accuracy_6: 0.0000e+00 - dense_2_accuracy_7: 0.0667 - dense_2_accuracy_8: 0.0667 - dense_2_accuracy_9: 0.0333 - dense_2_accuracy_10: 0.0500 - dense_2_accuracy_11: 0.0500 - dense_2_accuracy_12: 0.0667 - dense_2_accuracy_13: 0.0667 - dense_2_accuracy_14: 0.0667 - dense_2_accuracy_15: 0.0833 - dense_2_accuracy_16: 0.0833 - dense_2_accuracy_17: 0.0167 - dense_2_accuracy_18: 0.1000 - dense_2_accuracy_19: 0.0500 - dense_2_accuracy_20: 0.0500 - dense_2_accuracy_21: 0.0333 - dense_2_accuracy_22: 0.0500 - dense_2_accuracy_23: 0.1500 - dense_2_accuracy_24: 0.0667 - dense_2_accuracy_25: 0.0500 - dense_2_accuracy_26: 0.0667 - dense_2_accuracy_27: 0.0500 - dense_2_accuracy_28: 0.0500 - dense_2_accuracy_29:

60/60 [==============================] - 0s 1ms/step - loss: 96.0715 - dense_2_loss: 0.0000e+00 - dense_2_accuracy: 0.0667 - dense_2_accuracy_1: 0.1500 - dense_2_accuracy_2: 0.1667 - dense_2_accuracy_3: 0.1667 - dense_2_accuracy_4: 0.2167 - dense_2_accuracy_5: 0.1500 - dense_2_accuracy_6: 0.1500 - dense_2_accuracy_7: 0.2167 - dense_2_accuracy_8: 0.2000 - dense_2_accuracy_9: 0.1333 - dense_2_accuracy_10: 0.1500 - dense_2_accuracy_11: 0.1333 - dense_2_accuracy_12: 0.1167 - dense_2_accuracy_13: 0.2333 - dense_2_accuracy_14: 0.1500 - dense_2_accuracy_15: 0.1333 - dense_2_accuracy_16: 0.2000 - dense_2_accuracy_17: 0.1333 - dense_2_accuracy_18: 0.1500 - dense_2_accuracy_19: 0.1500 - dense_2_accuracy_20: 0.1333 - dense_2_accuracy_21: 0.1333 - dense_2_accuracy_22: 0.1000 - dense_2_accuracy_23: 0.0833 - dense_2_accuracy_24: 0.1333 - dense_2_accuracy_25: 0.2667 - dense_2_accuracy_26: 0.0667 - dense_2_accuracy_27: 0.2000 - dense_2_accuracy_28: 0.1667 - dense_2_accuracy_29: 0.0000e+00
Epoch 10/100

60/60 [==============================] - 0s 953us/step - loss: 63.4488 - dense_2_loss: 0.0000e+00 - dense_2_accuracy: 0.0667 - dense_2_accuracy_1: 0.2667 - dense_2_accuracy_2: 0.3500 - dense_2_accuracy_3: 0.2833 - dense_2_accuracy_4: 0.3500 - dense_2_accuracy_5: 0.3667 - dense_2_accuracy_6: 0.3500 - dense_2_accuracy_7: 0.4500 - dense_2_accuracy_8: 0.4333 - dense_2_accuracy_9: 0.4000 - dense_2_accuracy_10: 0.3667 - dense_2_accuracy_11: 0.4167 - dense_2_accuracy_12: 0.5333 - dense_2_accuracy_13: 0.5167 - dense_2_accuracy_14: 0.4333 - dense_2_accuracy_15: 0.4000 - dense_2_accuracy_16: 0.5833 - dense_2_accuracy_17: 0.4000 - dense_2_accuracy_18: 0.6000 - dense_2_accuracy_19: 0.4333 - dense_2_accuracy_20: 0.4333 - dense_2_accuracy_21: 0.4167 - dense_2_accuracy_22: 0.4833 - dense_2_accuracy_23: 0.4333 - dense_2_accuracy_24: 0.4333 - dense_2_accuracy_25: 0.5333 - dense_2_accuracy_26: 0.4500 - dense_2_accuracy_27: 0.5000 - dense_2_accuracy_28: 0.4833 - dense_2_accuracy_29: 0.0000e+00
Epoch 18/1

60/60 [==============================] - 0s 1ms/step - loss: 41.6166 - dense_2_loss: 0.0000e+00 - dense_2_accuracy: 0.0667 - dense_2_accuracy_1: 0.4000 - dense_2_accuracy_2: 0.4500 - dense_2_accuracy_3: 0.3667 - dense_2_accuracy_4: 0.5167 - dense_2_accuracy_5: 0.7833 - dense_2_accuracy_6: 0.7000 - dense_2_accuracy_7: 0.6333 - dense_2_accuracy_8: 0.7500 - dense_2_accuracy_9: 0.8000 - dense_2_accuracy_10: 0.7500 - dense_2_accuracy_11: 0.8000 - dense_2_accuracy_12: 0.9000 - dense_2_accuracy_13: 0.8500 - dense_2_accuracy_14: 0.7500 - dense_2_accuracy_15: 0.7833 - dense_2_accuracy_16: 0.8667 - dense_2_accuracy_17: 0.8500 - dense_2_accuracy_18: 0.8167 - dense_2_accuracy_19: 0.8667 - dense_2_accuracy_20: 0.7667 - dense_2_accuracy_21: 0.7500 - dense_2_accuracy_22: 0.8000 - dense_2_accuracy_23: 0.7000 - dense_2_accuracy_24: 0.6833 - dense_2_accuracy_25: 0.7333 - dense_2_accuracy_26: 0.8000 - dense_2_accuracy_27: 0.8333 - dense_2_accuracy_28: 0.7833 - dense_2_accuracy_29: 0.0000e+00
Epoch 26/100

60/60 [==============================] - 0s 1ms/step - loss: 26.1657 - dense_2_loss: 0.0000e+00 - dense_2_accuracy: 0.0333 - dense_2_accuracy_1: 0.4667 - dense_2_accuracy_2: 0.6167 - dense_2_accuracy_3: 0.6333 - dense_2_accuracy_4: 0.7667 - dense_2_accuracy_5: 0.8833 - dense_2_accuracy_6: 0.9833 - dense_2_accuracy_7: 0.9000 - dense_2_accuracy_8: 0.9667 - dense_2_accuracy_9: 0.9667 - dense_2_accuracy_10: 0.9667 - dense_2_accuracy_11: 1.0000 - dense_2_accuracy_12: 0.9833 - dense_2_accuracy_13: 0.9500 - dense_2_accuracy_14: 1.0000 - dense_2_accuracy_15: 1.0000 - dense_2_accuracy_16: 1.0000 - dense_2_accuracy_17: 1.0000 - dense_2_accuracy_18: 1.0000 - dense_2_accuracy_19: 0.9667 - dense_2_accuracy_20: 0.9833 - dense_2_accuracy_21: 0.9667 - dense_2_accuracy_22: 0.9833 - dense_2_accuracy_23: 0.9500 - dense_2_accuracy_24: 0.9167 - dense_2_accuracy_25: 0.9833 - dense_2_accuracy_26: 1.0000 - dense_2_accuracy_27: 0.9833 - dense_2_accuracy_28: 0.9000 - dense_2_accuracy_29: 0.0000e+00
Epoch 34/100

...

60/60 [==============================] - 0s 1ms/step - loss: 9.1649 - dense_2_loss: 0.0000e+00 - dense_2_accuracy: 0.0667 - dense_2_accuracy_1: 0.6167 - dense_2_accuracy_2: 0.8833 - dense_2_accuracy_3: 1.0000 - dense_2_accuracy_4: 1.0000 - dense_2_accuracy_5: 1.0000 - dense_2_accuracy_6: 1.0000 - dense_2_accuracy_7: 1.0000 - dense_2_accuracy_8: 1.0000 - dense_2_accuracy_9: 1.0000 - dense_2_accuracy_10: 1.0000 - dense_2_accuracy_11: 1.0000 - dense_2_accuracy_12: 1.0000 - dense_2_accuracy_13: 1.0000 - dense_2_accuracy_14: 1.0000 - dense_2_accuracy_15: 1.0000 - dense_2_accuracy_16: 1.0000 - dense_2_accuracy_17: 1.0000 - dense_2_accuracy_18: 1.0000 - dense_2_accuracy_19: 1.0000 - dense_2_accuracy_20: 1.0000 - dense_2_accuracy_21: 1.0000 - dense_2_accuracy_22: 1.0000 - dense_2_accuracy_23: 1.0000 - dense_2_accuracy_24: 1.0000 - dense_2_accuracy_25: 1.0000 - dense_2_accuracy_26: 1.0000 - dense_2_accuracy_27: 1.0000 - dense_2_accuracy_28: 1.0000 - dense_2_accuracy_29: 0.0000e+00
Epoch 66/100


60/60 [==============================] - 0s 1ms/step - loss: 8.8040 - dense_2_loss: 0.0000e+00 - dense_2_accuracy: 0.1000 - dense_2_accuracy_1: 0.6667 - dense_2_accuracy_2: 0.8833 - dense_2_accuracy_3: 1.0000 - dense_2_accuracy_4: 1.0000 - dense_2_accuracy_5: 1.0000 - dense_2_accuracy_6: 1.0000 - dense_2_accuracy_7: 1.0000 - dense_2_accuracy_8: 1.0000 - dense_2_accuracy_9: 1.0000 - dense_2_accuracy_10: 1.0000 - dense_2_accuracy_11: 1.0000 - dense_2_accuracy_12: 1.0000 - dense_2_accuracy_13: 1.0000 - dense_2_accuracy_14: 1.0000 - dense_2_accuracy_15: 1.0000 - dense_2_accuracy_16: 1.0000 - dense_2_accuracy_17: 1.0000 - dense_2_accuracy_18: 1.0000 - dense_2_accuracy_19: 1.0000 - dense_2_accuracy_20: 1.0000 - dense_2_accuracy_21: 1.0000 - dense_2_accuracy_22: 1.0000 - dense_2_accuracy_23: 1.0000 - dense_2_accuracy_24: 0.9667 - dense_2_accuracy_25: 1.0000 - dense_2_accuracy_26: 1.0000 - dense_2_accuracy_27: 1.0000 - dense_2_accuracy_28: 1.0000 - dense_2_accuracy_29: 0.0000e+00
Epoch 74/100


60/60 [==============================] - 0s 986us/step - loss: 8.2112 - dense_2_loss: 0.0000e+00 - dense_2_accuracy: 0.1000 - dense_2_accuracy_1: 0.6667 - dense_2_accuracy_2: 0.8833 - dense_2_accuracy_3: 1.0000 - dense_2_accuracy_4: 1.0000 - dense_2_accuracy_5: 1.0000 - dense_2_accuracy_6: 1.0000 - dense_2_accuracy_7: 1.0000 - dense_2_accuracy_8: 1.0000 - dense_2_accuracy_9: 1.0000 - dense_2_accuracy_10: 1.0000 - dense_2_accuracy_11: 1.0000 - dense_2_accuracy_12: 1.0000 - dense_2_accuracy_13: 1.0000 - dense_2_accuracy_14: 1.0000 - dense_2_accuracy_15: 1.0000 - dense_2_accuracy_16: 1.0000 - dense_2_accuracy_17: 1.0000 - dense_2_accuracy_18: 1.0000 - dense_2_accuracy_19: 1.0000 - dense_2_accuracy_20: 1.0000 - dense_2_accuracy_21: 1.0000 - dense_2_accuracy_22: 1.0000 - dense_2_accuracy_23: 1.0000 - dense_2_accuracy_24: 1.0000 - dense_2_accuracy_25: 1.0000 - dense_2_accuracy_26: 1.0000 - dense_2_accuracy_27: 1.0000 - dense_2_accuracy_28: 1.0000 - dense_2_accuracy_29: 0.0000e+00
Epoch 82/10

60/60 [==============================] - 0s 1ms/step - loss: 7.7782 - dense_2_loss: 0.0000e+00 - dense_2_accuracy: 0.1000 - dense_2_accuracy_1: 0.6667 - dense_2_accuracy_2: 0.8833 - dense_2_accuracy_3: 1.0000 - dense_2_accuracy_4: 1.0000 - dense_2_accuracy_5: 1.0000 - dense_2_accuracy_6: 1.0000 - dense_2_accuracy_7: 1.0000 - dense_2_accuracy_8: 1.0000 - dense_2_accuracy_9: 1.0000 - dense_2_accuracy_10: 1.0000 - dense_2_accuracy_11: 1.0000 - dense_2_accuracy_12: 1.0000 - dense_2_accuracy_13: 1.0000 - dense_2_accuracy_14: 1.0000 - dense_2_accuracy_15: 1.0000 - dense_2_accuracy_16: 1.0000 - dense_2_accuracy_17: 1.0000 - dense_2_accuracy_18: 1.0000 - dense_2_accuracy_19: 1.0000 - dense_2_accuracy_20: 1.0000 - dense_2_accuracy_21: 1.0000 - dense_2_accuracy_22: 1.0000 - dense_2_accuracy_23: 1.0000 - dense_2_accuracy_24: 1.0000 - dense_2_accuracy_25: 1.0000 - dense_2_accuracy_26: 1.0000 - dense_2_accuracy_27: 1.0000 - dense_2_accuracy_28: 1.0000 - dense_2_accuracy_29: 0.0000e+00
Epoch 90/100


60/60 [==============================] - 0s 1ms/step - loss: 7.5509 - dense_2_loss: 0.0000e+00 - dense_2_accuracy: 0.1000 - dense_2_accuracy_1: 0.7000 - dense_2_accuracy_2: 0.8833 - dense_2_accuracy_3: 1.0000 - dense_2_accuracy_4: 1.0000 - dense_2_accuracy_5: 1.0000 - dense_2_accuracy_6: 1.0000 - dense_2_accuracy_7: 1.0000 - dense_2_accuracy_8: 1.0000 - dense_2_accuracy_9: 1.0000 - dense_2_accuracy_10: 1.0000 - dense_2_accuracy_11: 1.0000 - dense_2_accuracy_12: 1.0000 - dense_2_accuracy_13: 1.0000 - dense_2_accuracy_14: 1.0000 - dense_2_accuracy_15: 1.0000 - dense_2_accuracy_16: 1.0000 - dense_2_accuracy_17: 1.0000 - dense_2_accuracy_18: 1.0000 - dense_2_accuracy_19: 1.0000 - dense_2_accuracy_20: 1.0000 - dense_2_accuracy_21: 1.0000 - dense_2_accuracy_22: 1.0000 - dense_2_accuracy_23: 1.0000 - dense_2_accuracy_24: 1.0000 - dense_2_accuracy_25: 1.0000 - dense_2_accuracy_26: 1.0000 - dense_2_accuracy_27: 1.0000 - dense_2_accuracy_28: 1.0000 - dense_2_accuracy_29: 0.0000e+00
Epoch 98/100


In [31]:
# GRADED FUNCTION: music_inference_model

def music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 100):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_values -- integer, umber of unique values
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """
    
    # Define the input of your model with a shape 
    x0 = Input(shape=(1, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0

    ### START CODE HERE ###
    # Step 1: Create an empty list of "outputs" to later store your predicted values (≈1 line)
    outputs = list()
    
    # Step 2: Loop over Ty and generate a value at every time step
    for t in range(Ty):
        
        # Step 2.A: Perform one step of LSTM_cell (≈1 line)
        a, _, c = LSTM_cell(x, initial_state = [a, c])
        
        # Step 2.B: Apply Dense layer to the hidden state output of the LSTM_cell (≈1 line)
        out = densor(a)

        # Step 2.C: Append the prediction "out" to "outputs". out.shape = (None, 78) (≈1 line)
        outputs.append(out)
        
        # Step 2.D: Select the next value according to "out", and set "x" to be the one-hot representation of the
        #           selected value, which will be passed as the input to LSTM_cell on the next step. We have provided 
        #           the line of code you need to do this. 
        x = Lambda(one_hot)(out)
        
    # Step 3: Create model instance with the correct "inputs" and "outputs" (≈1 line)
    inference_model = Model(inputs = [x0, a0, c0], outputs = outputs)
    
    ### END CODE HERE ###
    
    return inference_model

In [32]:
inference_model = music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 50)

In [33]:
# Check the inference model
inference_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 1, 78)        0                                            
__________________________________________________________________________________________________
a0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
c0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 64), (None,  36608       input_4[0][0]                    
                                                                 a0[0][0]                   

In [34]:
x_initializer = np.zeros((1, 1, 78))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

In [35]:
# GRADED FUNCTION: predict_and_sample

def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Predicts the next value of values using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference time
    x_initializer -- numpy array of shape (1, 1, 78), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    
    Returns:
    results -- numpy-array of shape (Ty, 78), matrix of one-hot vectors representing the values generated
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
    ### START CODE HERE ###
    # Step 1: Use your inference model to predict an output sequence given x_initializer, a_initializer and c_initializer.
    pred = inference_model.predict([x_initializer, a_initializer, c_initializer])
    # Step 2: Convert "pred" into an np.array() of indices with the maximum probabilities
    indices = np.argmax(pred, axis = -1)
    # Step 3: Convert indices to one-hot vectors, the shape of the results should be (Ty, n_values)
    results = to_categorical(indices, num_classes = 78)
    ### END CODE HERE ###
    
    return results, indices

In [36]:
results, indices = predict_and_sample(inference_model, x_initializer, a_initializer, c_initializer)
print("np.argmax(results[12]) =", np.argmax(results[12]))
print("np.argmax(results[17]) =", np.argmax(results[17]))
print("list(indices[12:18]) =", list(indices[12:18]))

np.argmax(results[12]) = 34
np.argmax(results[17]) = 24
list(indices[12:18]) = [array([34], dtype=int64), array([24], dtype=int64), array([26], dtype=int64), array([53], dtype=int64), array([34], dtype=int64), array([24], dtype=int64)]


In [37]:
out_stream = generate_music(inference_model)

Predicting new values for different set of chords.
Generated 50 sounds using the predicted values for the set of chords ("1") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("2") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("3") and after pruning
Generated 50 sounds using the predicted values for the set of chords ("4") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("5") and after pruning
Your generated music is saved in output/my_music.midi


In [38]:
IPython.display.Audio('./data/30s_trained_model.mp3')